In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.hidden1 = nn.Linear(7, 4)
        self.hidden2 = nn.Linear(4, 4)
        self.output = nn.Linear(4, 2)

    def forward(self, x):
        x = torch.sigmoid(self.hidden1(x))
        x = torch.sigmoid(self.hidden2(x))
        x = self.output(x)
        return x

# torch device if cuda exists
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load model from file 'model,epoch5,lr1e-3.bin'
lesser_model = MyNetwork().to(device)
lesser_model.load_state_dict(torch.load('model,epoch5,lr1e-3.bin'))
lesser_model.eval()

model = MyNetwork().to(device)
model.load_state_dict(torch.load('model,epoch50,lr1e-9.bin'))
model.eval()

eval_filename = 'test_data.csv'

In [3]:
from enum import Enum

class EduLevel(Enum):
    BACHELORS = "Bachelors"
    MASTERS = "Masters"
    PHD = "PHD"

edu_normalize = {
    EduLevel.BACHELORS: 0,
    EduLevel.MASTERS: 0.5,
    EduLevel.PHD: 1
}

payment_tiers_normalize = {
    "1": 0,
    "2": 0.5,
    "3": 1
}

experience_normalize = lambda x : x / 7
age_min = 22
age_max = 41
age_normalize = lambda x : (x - age_min) / (age_max - age_min)

import csv
from typing import Dict, Literal, List, Iterator

DatasetHeaders = Literal['Education', 'JoiningYear', 'City', 'PaymentTier', 'Age', 'Gender', 'EverBenched', 'ExperienceInCurrentDomain', 'LeaveOrNot']
DatasetRow = Dict[DatasetHeaders, List[str]]        

In [13]:
scores = [ 0, 0 ]
LESSER = 1
BOOM = 0
counter = 0
with open(eval_filename, 'r', newline='') as training_file:
    training_reader: Iterator[DatasetRow] = csv.DictReader(training_file)
    for i, row in enumerate(training_reader):
        counter += 1
        the_row = torch.tensor([
            edu_normalize[EduLevel(row['Education'])],
            payment_tiers_normalize[row['PaymentTier']],
            age_normalize(int(row['Age'])),
            1 if row['Gender'] == 'Male' else 0,
            1 if row['Gender'] == 'Female' else 0,
            1 if row['EverBenched'] == 'Yes' else 0,
            experience_normalize(int(row['ExperienceInCurrentDomain']))
        ]).float().to(device)
        
        boom = model(the_row).argmax().item()
        lesser_boom = lesser_model(the_row).argmax().item()
        
        if row['LeaveOrNot'] == '1':
            leave = 'yes'
        else:
            leave = 'no'
        
        if boom == 1:
            boom = 'no'
        else:
            boom = 'yes'
        
        if lesser_boom == 1:
            lesser_boom = 'no'
        else:
            lesser_boom = 'yes'
        
        if boom == leave:
            scores[BOOM] += 1
        
        if lesser_boom == leave:
            scores[LESSER] += 1
        
        # print(f"Truth: {leave}")
        # print(f"Lesser: {lesser_boom}")
        # print(f"Boom: {boom}\n")

In [23]:

# transform the data above into attractive table
import pandas as pd

df = pd.DataFrame({
    'Model': ['Boom', 'Lesser'],
    'Correct': [scores[BOOM], scores[LESSER]],
    'Total': [counter, counter],
    'Accuracy': [scores[BOOM] / counter, scores[LESSER] / counter],
    'Parameters': ['50 epochs, lr=1e-9', '5 epochs, lr=1e-3']
})

df

,Model,Correct,Total,Accuracy,Parameters
0,Boom,163,250,0.652,"50 epochs, lr=1e-9"
1,Lesser,162,250,0.648,"5 epochs, lr=1e-3"
